In [1]:
%display latex
import warnings
from IPython.display import display, Latex
render = lambda *args: show(*[(LatexExpr(a) if type(a) is str else latex(a))  for a in args])
hyp2exp = lambda expr: expr._maxima_().exponentialize().sage()
warnings.filterwarnings('ignore')
load('bch.sage')

#### We use the one-dimensional Poincaré-group in homogeneous coordinates to demonstrate a possible usage of the Baker-Campbell-Hausdorff formula.

*(The same can be calculated by hand with the [formula](https://en.wikipedia.org/wiki/Baker%E2%80%93Campbell%E2%80%93Hausdorff_formula#Selected_tractable_cases) given for the special case $ \left[X,Y\right] = s Y $ with appropriately choosing X and Y.)*

The Poincaré-transformation itself with the generators ($\chi$ is the rapidity):

In [2]:
var('chi', latex_name = '\chi '), var('t_0, x_0, t, x')
P = lambda chi, t_0, x_0: Matrix([[cosh(chi), sinh(chi), t_0],
                                  [sinh(chi), cosh(chi), x_0],
                                  [        0,         0,   1]])
Gen = lambda x: diff(P(chi, t_0, x_0), x).subs({x:0})
K , E, p = keep = -Gen(chi), -Gen(t_0), Gen(x_0)
X = Matrix([[t],[x],[1]])
render(r"P(\chi, t_0, x_0)", X, "=", P(chi,t_0,x_0)*X)
render("K = -\partial_\chi P(0,0,0) = ", K, ",~~ E = -\partial_{t_0} P(0,0,0) = ", E, ",~~ p = \partial_{x_0}P(0,0,0) = ", p)

P(\chi, t_0, x_0) \left(\begin{array}{r}
t \\
x \\
1
\end{array}\right) = \left(\begin{array}{r}
t \cosh\left({\chi }\right) + x \sinh\left({\chi }\right) + t_{0} \\
x \cosh\left({\chi }\right) + t \sinh\left({\chi }\right) + x_{0} \\
1
\end{array}\right)

K = -\partial_\chi P(0,0,0) =  \left(\begin{array}{rrr}
0 & -1 & 0 \\
-1 & 0 & 0 \\
0 & 0 & 0
\end{array}\right) ,~~ E = -\partial_{t_0} P(0,0,0) =  \left(\begin{array}{rrr}
0 & 0 & -1 \\
0 & 0 & 0 \\
0 & 0 & 0
\end{array}\right) ,~~ p = \partial_{x_0}P(0,0,0) =  \left(\begin{array}{rrr}
0 & 0 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0
\end{array}\right)

In [3]:
K*E-E*K == p, K*p-p*K == E, E*p == p*E

(True, True, True)

The group reconstructed as the product of one-parameter subgroups looks like the following:

$$\exp(-t_0 E)\exp(x_0 p)\exp(-\chi K) = \exp(-t_0 E + x_0 p) \exp(-\chi K) = \exp(X) \exp(Y),$$

where:

$$ X = -t_0 E + x_0 p, Y = - \chi K .$$

(The first two exponential can be trivially written as one because the two generators $E$, $p$ are already commuting). 
Sage can verify that this is the original matrix... well, at least partially:

In [4]:
render("\exp(X)\exp(Y) = ", (exp(-t_0*E + x_0*p)*exp(-chi*K)).canonicalize_radical()) #Sage cannot simplify this further easily.

\exp(X)\exp(Y) =  \left(\begin{array}{rrr}
\frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} + 1\right)} e^{\left(-{\chi }\right)} & \frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} - 1\right)} e^{\left(-{\chi }\right)} & t_{0} \\
\frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} - 1\right)} e^{\left(-{\chi }\right)} & \frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} + 1\right)} e^{\left(-{\chi }\right)} & x_{0} \\
0 & 0 & 1
\end{array}\right)

Now we can transform this further, as the whole group element can be expressed as an exponential of something, according to [BCH formula]( https://en.wikipedia.org/wiki/Baker–Campbell–Hausdorff_formula ):

$$ \exp(X) \exp(Y) = \exp(Z) = \exp( X + Y + \ldots) $$.

First we define an abstract Lie-algebra with the proper structure constants.

In [5]:
Poincare1D.<K,E,p> = LieAlgebra(SR, {('K','E'):{'p':1}, ('K','p'):{'E':1}})
comm = lambda x,y: render(r"\left[", x,  ",", y,r"\right] = ", Poincare1D[x,y])
comm(K,E)
comm(K,p)

\left[ K , E \right] =  p

\left[ K , p \right] =  E

Then calculate the [explicit BCH formula]( https://en.wikipedia.org/wiki/Baker%E2%80%93Campbell%E2%80%93Hausdorff_formula#An_explicit_Baker.E2.80.93Campbell.E2.80.93Hausdorff_formula ) until some order to be able to guess the shape of $Z$ defined above. Sage does the job and simplifies the repeated commutators.

In [6]:
X, Y = -t_0*E + x_0*p, -chi*K
Z = sum(Dynkin_BCH(X, Y, 6, commutator = Poincare1D.bracket))
render(r"Z \approx", Z)

Z \approx \left(-{\chi }\right)K + \left(-\frac{1}{30240} \, {\chi }^{6} t_{0} + \frac{1}{720} \, {\chi }^{4} t_{0} - \frac{1}{12} \, {\chi }^{2} t_{0} + \frac{1}{2} \, {\chi } x_{0} - t_{0}\right)E + \left(\frac{1}{30240} \, {\chi }^{6} x_{0} - \frac{1}{720} \, {\chi }^{4} x_{0} + \frac{1}{12} \, {\chi }^{2} x_{0} - \frac{1}{2} \, {\chi } t_{0} + x_{0}\right)p

We can identify the series-expansion appearing in the previous expression by looking up the sequence formed from the denominators of the coefficients: https://oeis.org/A227830.

In [7]:
f(chi) = -chi/(1-exp(chi))
render(f(chi) , r"\approx" , taylor(f(chi), chi, 0, 6))

\frac{{\chi }}{e^{\left({\chi }\right)} - 1} \approx \frac{1}{30240} \, {\chi }^{6} - \frac{1}{720} \, {\chi }^{4} + \frac{1}{12} \, {\chi }^{2} - \frac{1}{2} \, {\chi } + 1

Then we can figure out the formula for $Z$ and "prove" that it also gives back the same element of the matrix group we started from.

In [8]:
exponentialize = lambda K,E,p: exp(-chi*(K +(t_0-x_0)*(E+p)/2 - 1/(1-exp(chi))*(E*t_0 - p*x_0))) 
var('K', 'E','p')
render("P = ", exponentialize(K,E,p))

P =  e^{\left(-\frac{1}{2} \, {\left({\left(E + p\right)} {\left(t_{0} - x_{0}\right)} + 2 \, K + \frac{2 \, {\left(E t_{0} - p x_{0}\right)}}{e^{\left({\chi }\right)} - 1}\right)} {\chi }\right)}

In [9]:
K,E,p = keep
render(r"P(\chi, t_0, x_0) = ", exponentialize(K,E,p).canonicalize_radical())

P(\chi, t_0, x_0) =  \left(\begin{array}{rrr}
\frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} + 1\right)} e^{\left(-{\chi }\right)} & \frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} - 1\right)} e^{\left(-{\chi }\right)} & t_{0} \\
\frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} - 1\right)} e^{\left(-{\chi }\right)} & \frac{1}{2} \, {\left(e^{\left(2 \, {\chi }\right)} + 1\right)} e^{\left(-{\chi }\right)} & x_{0} \\
0 & 0 & 1
\end{array}\right)